In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
peft_model_id = "ybelkada/opt-350m-lora"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(peft_model_id)

In [ ]:
model_id = "facebook/opt-350m"
peft_model_id = "ybelkada/opt-350m-lora"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id)
model.load_adapter(peft_model_id)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(peft_model_id, quantization_config=BitsAndBytesConfig(load_in_8bit=True), low_cpu_mem_usage=True)

In [ ]:
from transformers import AutoModelForCausalLM, OPTForCausalLM, AutoTokenizer
from peft import LoraConfig

In [ ]:
model_id = "facebook/opt-350m"
model = AutoModelForCausalLM.from_pretrained(model_id)

In [ ]:
lora_config = LoraConfig(
    target_modules=["q_proj", "k_proj"],
    init_lora_weights=False
)

In [ ]:
model.add_adapter(lora_config, adapter_name="adapter_1")

In [ ]:
model.add_adapter(lora_config, adapter_name="adapter_2")

In [ ]:
# use adapter_1
model.set_adapter("adapter_1")
output = model.generate(**inputs)
print(tokenizer.decode(output_disabled[0], skip_special_tokens=True))

In [ ]:
# use adapter_2
model.set_adapter("adapter_2")
output_enabled = model.generate(**inputs)
print(tokenizer.decode(output_enabled[0], skip_special_tokens=True))

In [ ]:
from transformers import AutoModelForCausalLM, OPTForCausalLM, AutoTokenizer
from peft import PeftConfig

model_id = "facebook/opt-350m"
adapter_model_id = "ybelkada/opt-350m-lora"
tokenizer = AutoTokenizer.from_pretrained(model_id)
text = "Hello"
inputs = tokenizer(text, return_tensors="pt")

model = AutoModelForCausalLM.from_pretrained(model_id)
peft_config = PeftConfig.from_pretrained(adapter_model_id)

# to initiate with random weights
peft_config.init_lora_weights = False

model.add_adapter(peft_config)
model.enable_adapters()
output = model.generate(**inputs)

In [ ]:
output

In [ ]:
tokenizer.decode(output[0])

In [ ]:
model.disable_adapters()
output = model.generate(**inputs)

In [ ]:
tokenizer.decode(output[0])

# Train PEFT Adapter

In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("yelp_review_full")

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
from transformers import TrainingArguments, Trainer


In [ ]:
training_args = TrainingArguments(output_dir="test_trainer")


In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
model_id = "facebook/opt-350m"
model = AutoModelForCausalLM.from_pretrained(model_id)

lora_config = LoraConfig(
    target_modules=["q_proj", "k_proj"],
    modules_to_save=["lm_head"],
)

model.add_adapter(lora_config)

In [ ]:
output_enabled = model.generate(**inputs)
print(tokenizer.decode(output_enabled[0], skip_special_tokens=True))

In [ ]:
model.disable_adapters()

In [ ]:
output_enabled = model.generate(**inputs)
print(tokenizer.decode(output_enabled[0], skip_special_tokens=True))

# BERT Example

In [1]:
from datasets import concatenate_datasets, load_dataset
from transformers import BertTokenizerFast, AutoTokenizer
import multiprocessing
from tqdm import tqdm
from itertools import chain

In [2]:
dataset = load_dataset("yelp_review_full")

In [3]:
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(10000))
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(10000))

In [4]:
# repositor id for saving the tokenizer
tokenizer_id="nlptown/bert-base-multilingual-uncased-sentiment"

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors="pt")

In [6]:
# create a tokenizer from existing one to re-use special tokens
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)

/home/geshi/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
tokenized_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)

In [8]:
print(f"the dataset contains in total {len(tokenized_train_dataset)*tokenizer.model_max_length} tokens")

the dataset contains in total 5120000 tokens


# Create Model

In [9]:
from transformers import AutoModelForPreTraining, DataCollatorForTokenClassification, TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AdamW, get_scheduler
import evaluate

/home/geshi/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [10]:
pretrained_model_id = 'distilbert-base-uncased'

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_id, num_labels=5)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# LoRA configuration
lora_config = LoraConfig(
    task_type="SEQ_CLS",
    inference_mode=False,
    r=8,  # Rank
    lora_alpha=16,  # LoRA scaling
    lora_dropout=0.1,  # Dropout for LoRA layers
    target_modules="all-linear",
    bias="none",
)

In [13]:
model = get_peft_model(model, lora_config)

In [14]:
model.print_trainable_parameters() # see % trainable parameters

trainable params: 1,276,461 || all params: 68,252,250 || trainable%: 1.8702


In [15]:
training_args = TrainingArguments(
    'test-trainer',
    evaluation_strategy="epoch",
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 5,
    learning_rate = 2e-5,
    weight_decay = 0.01,
    seed=42,
)

/home/geshi/.local/lib/python3.10/site-packages/transformers/training_args.py:1528: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
metric = evaluate.load("accuracy")

In [17]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
data_collator = DataCollatorWithPadding(tokenizer)

In [19]:
tokenized_train_dataset[0]['input_ids']

[101,
 151,
 16104,
 10167,
 10372,
 38325,
 119,
 151,
 112,
 10312,
 10662,
 10114,
 15149,
 17837,
 10982,
 151,
 60066,
 11408,
 10114,
 10346,
 143,
 26097,
 48695,
 24368,
 10104,
 11585,
 117,
 29676,
 32332,
 23523,
 71576,
 117,
 10110,
 10108,
 13039,
 10103,
 36799,
 112,
 161,
 15200,
 119,
 10103,
 21170,
 10107,
 10320,
 10297,
 10297,
 35925,
 10563,
 34086,
 119,
 10103,
 23661,
 10127,
 12278,
 35751,
 10110,
 10191,
 19552,
 11885,
 21170,
 10151,
 13455,
 10171,
 10144,
 81981,
 24205,
 10108,
 29152,
 119,
 10855,
 93954,
 39999,
 14048,
 67922,
 106,
 106,
 106,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [20]:
tokenized_train_dataset['label']

[4,
 2,
 4,
 0,
 2,
 2,
 3,
 0,
 0,
 4,
 0,
 4,
 3,
 1,
 3,
 0,
 4,
 3,
 4,
 2,
 0,
 4,
 0,
 1,
 0,
 1,
 1,
 2,
 4,
 3,
 3,
 2,
 4,
 3,
 0,
 2,
 2,
 0,
 1,
 2,
 2,
 2,
 0,
 1,
 4,
 1,
 1,
 2,
 0,
 2,
 1,
 1,
 2,
 4,
 1,
 3,
 2,
 4,
 3,
 3,
 3,
 1,
 0,
 3,
 3,
 2,
 1,
 0,
 0,
 0,
 3,
 2,
 0,
 3,
 4,
 4,
 0,
 1,
 3,
 1,
 0,
 3,
 1,
 4,
 1,
 3,
 2,
 3,
 2,
 4,
 3,
 3,
 1,
 4,
 0,
 2,
 4,
 1,
 4,
 3,
 3,
 3,
 0,
 2,
 4,
 0,
 4,
 4,
 2,
 1,
 4,
 3,
 4,
 1,
 2,
 0,
 3,
 0,
 0,
 0,
 4,
 3,
 1,
 0,
 3,
 4,
 0,
 4,
 1,
 4,
 0,
 3,
 0,
 1,
 2,
 3,
 0,
 1,
 2,
 2,
 2,
 1,
 0,
 2,
 3,
 4,
 2,
 0,
 1,
 1,
 3,
 1,
 1,
 0,
 3,
 2,
 4,
 3,
 3,
 4,
 4,
 0,
 2,
 3,
 4,
 1,
 1,
 4,
 3,
 2,
 1,
 1,
 3,
 3,
 4,
 3,
 1,
 0,
 4,
 2,
 1,
 4,
 3,
 4,
 1,
 0,
 0,
 1,
 4,
 4,
 3,
 2,
 0,
 2,
 3,
 0,
 3,
 1,
 3,
 4,
 1,
 1,
 2,
 2,
 2,
 2,
 3,
 0,
 0,
 2,
 3,
 3,
 3,
 3,
 4,
 3,
 0,
 3,
 4,
 3,
 3,
 1,
 4,
 0,
 0,
 2,
 3,
 3,
 4,
 0,
 4,
 4,
 4,
 0,
 2,
 1,
 2,
 0,
 2,
 3,
 4,
 3,
 1,
 1,
 4,
 0,
 1,
 3,
 1,
 0,


In [21]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [22]:
trainer = Trainer(
    model,
    args=training_args,
    train_dataset=tokenized_train_dataset, # training dataset requires column input_ids
    eval_dataset=tokenized_eval_dataset,
    compute_metrics=compute_metrics,
    # data_collator=data_collator,
    # tokenizer=tokenizer
)

In [ ]:
trainer.train()

[2024-08-22 17:57:02,269] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.4
 [WARNING]  using untested triton version (3.0.0), only 1.0.0 is known to be compatible


/home/geshi/.local/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:49: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, weight, bias=None):
/home/geshi/.local/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctr

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/geshi/.netrc
